In [10]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=90abb5ff948578adcaf36c66e8c7390c81e3205124857e90d2feec0430b8cdbc
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=f816f27a8bd38f6533ac148d6bc242ceb3f2687d7a8a6388ebc10e553fecbd47
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

2.3.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [8]:
def build_model (hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'] )
  
  return model
  

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search=RandomSearch(build_model, objective='val_accuracy', max_trials=5, 
                          directory='output', project_name="MNIST Fashion")

In [13]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.7561 - accuracy: 0.7590 - val_loss: 0.4486 - val_accuracy: 0.8340
Epoch 2/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.4723 - accuracy: 0.8281 - val_loss: 0.5027 - val_accuracy: 0.8113
Epoch 3/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.4737 - accuracy: 0.8238 - val_loss: 0.4388 - val_accuracy: 0.8327


Epoch 1/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.3667 - accuracy: 0.8684 - val_loss: 0.2834 - val_accuracy: 0.8948
Epoch 2/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.2198 - accuracy: 0.9172 - val_loss: 0.2549 - val_accuracy: 0.9058
Epoch 3/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.1491 - accuracy: 0.9446 - val_loss: 0.2431 - val_accuracy: 0.9115


Epoch 1/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.4128 - accuracy: 0.8503 - val_loss: 0.3242 - val_accuracy: 0.8750
Epoch 2/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.2699 - accuracy: 0.9007 - val_loss: 0.2692 - val_accuracy: 0.9010
Epoch 3/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.2196 - accuracy: 0.9188 - val_loss: 0.2635 - val_accuracy: 0.9027


Epoch 1/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.5182 - accuracy: 0.8336 - val_loss: 0.3843 - val_accuracy: 0.8647
Epoch 2/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.3436 - accuracy: 0.8731 - val_loss: 0.3482 - val_accuracy: 0.8672
Epoch 3/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.3013 - accuracy: 0.8892 - val_loss: 0.3342 - val_accuracy: 0.8808


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.5478 - accuracy: 0.8157 - val_loss: 0.4210 - val_accuracy: 0.8435
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3976 - accuracy: 0.8532 - val_loss: 0.4128 - val_accuracy: 0.8528
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3638 - accuracy: 0.8661 - val_loss: 0.4055 - val_accuracy: 0.8580


INFO:tensorflow:Oracle triggered exit


In [16]:
model = tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 96)        83040     
_________________________________________________________________
flatten (Flatten)            (None, 55296)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               7078016   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 7,163,306
Trainable params: 7,163,306
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0952 - accuracy: 0.9647 - val_loss: 0.2732 - val_accuracy: 0.9145
Epoch 5/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0583 - accuracy: 0.9790 - val_loss: 0.3926 - val_accuracy: 0.9123
Epoch 6/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0409 - accuracy: 0.9856 - val_loss: 0.3591 - val_accuracy: 0.9120
Epoch 7/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0282 - accuracy: 0.9898 - val_loss: 0.4332 - val_accuracy: 0.9138
Epoch 8/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0220 - accuracy: 0.9924 - val_loss: 0.5421 - val_accuracy: 0.9095
Epoch 9/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0210 - accuracy: 0.9929 - val_loss: 0.4945 - val_accuracy: 0.9140
Epoch 10/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0172 - accuracy: 0.9943 - val_loss: 0.5973 - val_a